<h1 style="color:#1B4F72;">UANL OCR</h1>
<b>Pedro Antonio Martinez Gonzalez 1798942</b>
<br><br>
<i>El objetivo es procesar 1,110 archivos PDF todos escaneados, extraer la matricula y nombre del alumno para posteriormente modificar el nombre del archivo PDF por la matricula y nombre.</i>

<h1 style="color:#1B4F72;">Instalar Dependencias</h1>
<h3>Ejecutar los siguientes comandos</h3>
<i>pip install pdf2image</i><br>
<i>pip install opencv-python</i><br>
<i>pip install pytesseract</i><br>

<h3>Instalar pyteseract en windows</h3>
<i>Descargar el ejecutable de </i><a>https://github.com/UB-Mannheim/tesseract/wiki</a><i>, es recomendable instalarlo para todos los usuarios del sistema y en la ruta predeterminada de instalación, de lo contrario será necesario cambiar la línea 15 del main.py por la nueva ruta de tesseract.exe.</i>

In [4]:
from pdf2image import convert_from_path
import cv2
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
from os import scandir
import numpy as np
import re
import os

banned_symbol = ('/','|','"','!','(',')','_',';','.','~','‘','-','“','—',"'",'*',':','[',']','»',',','}','{','§','°','=')
banned_letter = ('a', 'b', 'c', 'd', 'e', 'é', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z')
banned_number = ('0','1','2','3','4','5','6','7','8','9')

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 6'

<h3>Esta función devuelve una lista con los archivos contenidos en un directorio</h3>

In [5]:
def ls2(path): 
    return [obj.name for obj in scandir(path) if obj.is_file()]

<h3>Convierte un archivo PDF a una Imagen</h3>

In [6]:
def pdf_to_image(pdf_name):
    images = convert_from_path(pdf_path = 'PDF/' + pdf_name , poppler_path = r'poppler-21.02.0\Library\bin')
    images[0].save(pdf_name + '.jpg', 'JPEG')

<img src=https://pythonforundergradengineers.com/posts/pdf/images/pdf_to_dir.png width=504 height=274>

<h3>Esta función preparar la imagen para ser procesada por el OCR</h3>

In [3]:
def image_pre_process(jpg_name):
    image = cv2.imread(jpg_name + '.jpg')
    os.remove(jpg_name + '.jpg')
    
    matricula_img = ''
    name_img = ''
    x = False
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config, lang='eng')
    total_boxes = len(details['text'])
    
    for sequence_number in range(total_boxes):
        if details['text'][sequence_number] == "Alumno" and x == False:
            if int(details['conf'][sequence_number]) >30:
                (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number], details['height'][sequence_number])
                matricula_img = threshold_img[y - 10 : y + h + 60 , x - 45 : x + w + 50]
                x = True
                continue

        if details['text'][sequence_number] == "Alumno" and x == True:
            if int(details['conf'][sequence_number]) >30:
                (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number], details['height'][sequence_number])
                name_img = threshold_img[y + 12 : y + h + 65 , x - 150 : x + w + 600]

    return matricula_img, name_img

<img src="https://scontent.fmty1-1.fna.fbcdn.net/v/t1.15752-9/164340173_210942620825075_1228154719396614310_n.png?_nc_cat=110&ccb=1-3&_nc_sid=ae9488&_nc_ohc=1LQmyAHbBmUAX8XficG&_nc_oc=AQm9Y7YZFlGVNb2ebqRJHzGRjkRIBNt8LMDPrn7-PpnbYzTT5By3HAPqgxH8xm8di7I&_nc_ht=scontent.fmty1-1.fna&oh=3db28e53ea5b6ac114da29427eb9c330&oe=60884F52">

<h3>Procesa la imagen y extrae los valores de matricula y nombre</h3>

In [7]:
def image_process(matricula_img, name_img, file):
    name = ''
    matricula = ''
    
    if type(matricula_img).__module__ == np.__name__ and type(name_img).__module__ == np.__name__:
        matricula_details = pytesseract.image_to_data(matricula_img, output_type=Output.DICT, config=custom_config, lang='eng')
        name_details = pytesseract.image_to_data(name_img, output_type=Output.DICT, config=custom_config, lang='eng')

        for i in range(len(matricula_details['text'])):
            if re.findall("\d{7}", matricula_details['text'][i]) and matricula == '':
                matricula = matricula_details['text'][i]

                if int(len(matricula)) >= 8:
                    matricula = matricula[:7]
    
        for a in range(len(name_details['text'])):
            name = name + name_details['text'][a]

        for symbol in banned_number:
            name = name.replace(symbol , '')

        name = name.replace(",", " ")
        new_file_name = matricula + " " +  name
    
    else:
        file = file.replace('.pdf','')
        new_file_name = 'ERROR ' + file
    
    return new_file_name

<h3>Esta función cambia el nombre del archivo</h3>

In [8]:
def change_name(old_file_name, new_file_name):
    try:
        os.rename('PDF/{}'.format(old_file_name), 
                'PDF/{}.pdf'.format(new_file_name))
    except:
        os.rename('PDF/{}'.format(old_file_name), 
                'PDF/{}'.format('ERROR ' + old_file_name))

<h3>Aquí se ejecutan todas las funciones definidas anteriormente</h3>

In [ ]:
files = ls2('PDF/')

for file in files:
    print("Archivo de Entrada:\t" + file)

    pdf_to_image(file)
    
    matricula_img, name_img = image_pre_process(file)
    new_file_name = image_process(matricula_img, name_img, file)

    for symbol in banned_symbol:
        new_file_name = new_file_name.replace(symbol , '')
    for symbol in banned_letter:
        new_file_name = new_file_name.replace(symbol , '')

    change_name(file, new_file_name)

    print("Archivo de Salida:\t" + new_file_name + ".pdf\n")

Archivo de Entrada:	1449328 GONZALEZ CHARLES ANTONIO.pdf
Archivo de Salida:	1449328 GONZALEZ CHARLES ANTONIO.pdf

Archivo de Entrada:	1450903 GARCIA ZUNIGA ALAN.pdf
Archivo de Salida:	1450903 GARCIA ZUNIGA ALAN.pdf

Archivo de Entrada:	1456670 ORTIZ MORALES CRISTOPHERVIDAL.pdf
Archivo de Salida:	1456670 ORTIZ MORALES CRISTOPHERVIDAL.pdf

Archivo de Entrada:	1488970 MORENO CORONADO OSCARANDRES.pdf
Archivo de Salida:	1488970 MORENO CORONADO OSCARANDRES.pdf

Archivo de Entrada:	1497040 TORRES GONZALEZ NELLYMICHELL.pdf
Archivo de Salida:	1497040 TORRES GONZALEZ NELLYMICHELL.pdf

Archivo de Entrada:	1594693 MARTINEZ MENDEZ ALEJANDRO.pdf


<h1 style="color:#1B4F72;"> Ejecucion</h1>
<i>Para la ejecución es necesario cumplir con las dependencias de la sección “Instalar Dependencias”, lo siguiente es colocar los archivos que desea renombrar en el directorio “PDF”, por último, ejecutar el main.py con el comando “python main.py”</i>
<img src="https://scontent-qro1-1.xx.fbcdn.net/v/t1.15752-9/155914620_1043209652838958_216827113365319928_n.png?_nc_cat=109&ccb=3&_nc_sid=ae9488&_nc_eui2=AeHSaZFZqTvy6SIF5HVJB7zuVfXPyRnOtnlV9c_JGc62ebO4MN5YL_optt8fB3HMFXt4Mmb-qErUpp_GFtVMT7Wb&_nc_ohc=YddR4pbSCYYAX8-SrA5&_nc_ht=scontent-qro1-1.xx&oh=c34adff9c8343a0e0155c32432007596&oe=6062FDD0">